#### tf serving makes model version management and model serving very easy

it has batch inferences : it will batch the requests for better hardware utilization

here we have some random model

In [28]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [5]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

In [6]:
sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

vocab_size = 40

encoded_reviews = [one_hot(a, vocab_size) for a in reviews]

max_length = 3

padded_reviews = pad_sequences(encoded_reviews, maxlen= max_length, padding= 'post')

embeded_vector_size = 4

model = Sequential()
model.add(Embedding(vocab_size, embeded_vector_size, input_length=max_length, name= 'embedding'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

x = padded_reviews
y = sentiment

model.fit(x, y, epochs=100, verbose=0)

save the model 3 time to use it as version control in tf serving

In [6]:
model.save('saved_models/1/')

INFO:tensorflow:Assets written to: saved_models/1/assets


In [6]:
model.save('saved_models/2/')

INFO:tensorflow:Assets written to: saved_models/2/assets


In [ ]:
model.save('saved_models/3/')

# quantization

reduce the model size to use it on other devices

quantization is a process of reducing model size so that it can run on EDGE devices

converting all weights numbers float into integer to save memory with some algorithms that model result wont change that much

benefits are: run ML models on every devices EDGE , faster inference

performing: 

1. post training quantization

       use tf.lite convert

2. quantization aware training

       use quantize_model function on model then do training again then use tf.lite , accuracy wont suffer

In [8]:
model.save('./saved_models')

INFO:tensorflow:Assets written to: ./saved_models\assets


85

### 1- post training quantization

In [29]:
import tensorflow as tf

In [19]:
convertor = tf.lite.TFLiteConverter.from_saved_model('.\saved_models')

tflite_model = convertor.convert()

In [20]:
len(tflite_model)

2676

In [21]:
convertor = tf.lite.TFLiteConverter.from_saved_model('.\saved_models')
convertor.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_quant_model = convertor.convert()

In [23]:
len(tflite_quant_model)

2680

In [24]:
with open('tf_model.tflite', 'wb') as f:
    f.write(tflite_model)

with open('tf_quant_model.tflite', 'wb') as f:
    f.write(tflite_quant_model)